# 08. Correlation Analysis: News Sentiment and Stock Movement

This notebook analyzes the correlation between news sentiment and stock price movements.

**Sections:**
1. **Data Loading and Preparation** - Load news and stock data, align dates
2. **Sentiment Analysis** - Perform sentiment analysis on headlines using NLTK, TextBlob, and VADER
3. **Stock Movement Calculation** - Calculate daily stock returns
4. **Correlation Analysis** - Aggregate daily sentiments and calculate correlation with stock returns

**Dependencies:** 
- Run `01_Data_Loading_and_Setup.ipynb` first for processed news data
- Run `07_Quantitative_Analysis.ipynb` for stock price data (optional)
- Stock price data files in `data/Data/Data/` directory

**Stocks Analyzed:** AAPL, AMZN, GOOG, META, MSFT, NVDA


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Sentiment Analysis libraries
import nltk
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Statistical Analysis
from scipy import stats
from scipy.stats import pearsonr

# Add utils to path
sys.path.append('.')
from utils import setup_plotting_style, load_processed_data

# Setup plotting style
setup_plotting_style()

# Download required NLTK data
print("Downloading NLTK data...")
try:
    try:
        nltk.download('punkt_tab', quiet=True)
        print("Downloaded punkt_tab")
    except Exception as e1:
        try:
            nltk.download('punkt', quiet=True)
            print("Downloaded punkt (fallback)")
        except Exception as e2:
            print(f"Warning: Could not download punkt: {e2}")
    
    try:
        nltk.download('stopwords', quiet=True)
        print("Downloaded stopwords")
    except Exception as e:
        print(f"Warning: Could not download stopwords: {e}")
    
    print("NLTK data ready!")
except Exception as e:
    print(f"NLTK download error: {e}")

# Initialize VADER sentiment analyzer
vader_analyzer = SentimentIntensityAnalyzer()

print("\nLibraries imported successfully!")


## 1. Data Loading and Preparation

**Objective:** Load news and stock price data, then align dates to ensure each news item matches the corresponding stock trading day.

**Key Steps:**
- Load processed news data
- Load stock price data for all stocks
- Normalize timestamps and align dates
- Handle missing dates and trading day matching


In [ ]:
# Load processed news data
print("="*80)
print("LOADING NEWS DATA")
print("="*80)

try:
    df_news = load_processed_data('../data/processed/df_processed.pkl')
    print(f"Loaded processed news data: {df_news.shape}")
except FileNotFoundError:
    print("Processed data not found. Loading raw data...")
    from utils import load_data, preprocess_data, clean_data
    df_news = load_data('../data/raw_analyst_ratings.csv')
    df_news = clean_data(df_news, remove_duplicates=True, handle_missing='report', fix_dates=True)
    df_news = preprocess_data(df_news)
    print(f"Loaded and preprocessed news data: {df_news.shape}")

print(f"\nNews dataset shape: {df_news.shape}")
print(f"Columns: {list(df_news.columns)}")

# Ensure date is datetime
if not pd.api.types.is_datetime64_any_dtype(df_news['date']):
    df_news['date'] = pd.to_datetime(df_news['date'], errors='coerce')

# Extract date only (without time) for alignment
df_news['date_only'] = df_news['date'].dt.date

# Filter out invalid dates
df_news_valid = df_news[df_news['date'].notna()].copy()
print(f"\nValid news dates: {len(df_news_valid):,} ({len(df_news_valid)/len(df_news)*100:.2f}%)")
print(f"Date range: {df_news_valid['date'].min()} to {df_news_valid['date'].max()}")


In [ ]:
# Load stock price data
print("\n" + "="*80)
print("LOADING STOCK PRICE DATA")
print("="*80)

STOCKS = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
DATA_PATH = '../data/Data/Data/'

stock_data = {}

for stock in STOCKS:
    file_path = f"{DATA_PATH}{stock}.csv"
    try:
        df_stock = pd.read_csv(file_path)
        
        # Convert Date column to datetime
        if 'Date' in df_stock.columns:
            df_stock['Date'] = pd.to_datetime(df_stock['Date'])
            df_stock = df_stock.sort_values('Date').reset_index(drop=True)
            df_stock.set_index('Date', inplace=True)
            df_stock.index.name = 'date'
        
        # Extract date only
        df_stock['date_only'] = df_stock.index.date
        
        # Verify required columns
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        if all(col in df_stock.columns for col in required_cols):
            stock_data[stock] = df_stock
            print(f"✓ {stock}: Loaded {len(df_stock):,} rows, Date range: {df_stock.index.min()} to {df_stock.index.max()}")
        else:
            print(f"⚠ {stock}: Missing required columns")
    except FileNotFoundError:
        print(f"✗ {stock}: File not found at {file_path}")
    except Exception as e:
        print(f"✗ {stock}: Error loading data - {e}")

print(f"\nSuccessfully loaded {len(stock_data)} stock datasets")


In [ ]:
# Date alignment: Normalize timestamps and match to trading days
print("\n" + "="*80)
print("DATE ALIGNMENT")
print("="*80)

# For news data: Convert to date only and ensure we're working with trading days
# Stock markets are typically closed on weekends, so we need to match news to trading days

# Get all trading days from stock data (union of all stock trading days)
all_trading_days = set()
for stock, df_stock in stock_data.items():
    trading_days = set(df_stock.index.date)
    all_trading_days.update(trading_days)

all_trading_days = sorted(list(all_trading_days))
print(f"Total unique trading days across all stocks: {len(all_trading_days):,}")
print(f"Trading day range: {all_trading_days[0]} to {all_trading_days[-1]}")

# Filter news data to only include dates that are trading days
df_news_valid['is_trading_day'] = df_news_valid['date_only'].isin(all_trading_days)
news_on_trading_days = df_news_valid[df_news_valid['is_trading_day']].copy()

print(f"\nNews articles on trading days: {len(news_on_trading_days):,} ({len(news_on_trading_days)/len(df_news_valid)*100:.2f}%)")
print(f"News articles on non-trading days: {(~df_news_valid['is_trading_day']).sum():,}")

# For correlation analysis, we'll use news on trading days
df_news_aligned = news_on_trading_days.copy()
print(f"\n✓ News data aligned to trading days: {len(df_news_aligned):,} articles")


## 2. Sentiment Analysis

**Objective:** Perform sentiment analysis on headlines using multiple tools (NLTK, TextBlob, VADER) to quantify the tone of each article.

**Tools Used:**
- **TextBlob**: Polarity score (-1 to 1, where -1 is negative, 1 is positive)
- **VADER**: Compound score (-1 to 1, optimized for social media and financial text)
- **NLTK**: For text preprocessing

**Key Questions:**
- What is the distribution of sentiment scores?
- How do different sentiment analyzers compare?
- Are there differences in sentiment by stock or publisher?


In [ ]:
# Perform sentiment analysis on headlines
print("="*80)
print("SENTIMENT ANALYSIS")
print("="*80)

# For efficiency, we'll analyze a sample if the dataset is very large
# You can adjust this or remove the sampling for full analysis
SAMPLE_SIZE = None  # Set to None for full analysis, or a number like 100000 for sampling

if SAMPLE_SIZE and len(df_news_aligned) > SAMPLE_SIZE:
    print(f"Sampling {SAMPLE_SIZE:,} articles for sentiment analysis...")
    df_sentiment = df_news_aligned.sample(n=SAMPLE_SIZE, random_state=42).copy()
else:
    df_sentiment = df_news_aligned.copy()

print(f"Analyzing sentiment for {len(df_sentiment):,} headlines...")

# Initialize sentiment analyzers
vader_analyzer = SentimentIntensityAnalyzer()

# Calculate sentiment scores
def calculate_textblob_sentiment(text):
    """Calculate TextBlob sentiment polarity"""
    try:
        blob = TextBlob(str(text))
        return blob.sentiment.polarity
    except:
        return 0.0

def calculate_vader_sentiment(text):
    """Calculate VADER sentiment compound score"""
    try:
        scores = vader_analyzer.polarity_scores(str(text))
        return scores['compound']
    except:
        return 0.0

# Apply sentiment analysis
print("\nCalculating TextBlob sentiment...")
df_sentiment['sentiment_textblob'] = df_sentiment['headline'].apply(calculate_textblob_sentiment)

print("Calculating VADER sentiment...")
df_sentiment['sentiment_vader'] = df_sentiment['headline'].apply(calculate_vader_sentiment)

# Create a combined sentiment score (average of both)
df_sentiment['sentiment_combined'] = (df_sentiment['sentiment_textblob'] + df_sentiment['sentiment_vader']) / 2

# Classify sentiment
def classify_sentiment(score):
    """Classify sentiment as positive, negative, or neutral"""
    if score > 0.1:
        return 'positive'
    elif score < -0.1:
        return 'negative'
    else:
        return 'neutral'

df_sentiment['sentiment_label'] = df_sentiment['sentiment_combined'].apply(classify_sentiment)

print(f"\n✓ Sentiment analysis complete for {len(df_sentiment):,} headlines")


In [ ]:
# Analyze sentiment distribution
print("\n" + "="*80)
print("SENTIMENT DISTRIBUTION ANALYSIS")
print("="*80)

print(f"\nSentiment Classification (Combined Score):")
sentiment_counts = df_sentiment['sentiment_label'].value_counts()
for label, count in sentiment_counts.items():
    pct = count / len(df_sentiment) * 100
    print(f"  {label:10s}: {count:>8,} ({pct:>5.2f}%)")

print(f"\nSentiment Score Statistics (Combined):")
print(f"  Mean:   {df_sentiment['sentiment_combined'].mean():.4f}")
print(f"  Median: {df_sentiment['sentiment_combined'].median():.4f}")
print(f"  Std:    {df_sentiment['sentiment_combined'].std():.4f}")
print(f"  Min:    {df_sentiment['sentiment_combined'].min():.4f}")
print(f"  Max:    {df_sentiment['sentiment_combined'].max():.4f}")

print(f"\nSentiment Score Statistics (TextBlob):")
print(f"  Mean:   {df_sentiment['sentiment_textblob'].mean():.4f}")
print(f"  Median: {df_sentiment['sentiment_textblob'].median():.4f}")

print(f"\nSentiment Score Statistics (VADER):")
print(f"  Mean:   {df_sentiment['sentiment_vader'].mean():.4f}")
print(f"  Median: {df_sentiment['sentiment_vader'].median():.4f}")

# Sentiment by stock
print(f"\n" + "="*80)
print("SENTIMENT BY STOCK")
print("="*80)
for stock in STOCKS:
    stock_news = df_sentiment[df_sentiment['stock'] == stock]
    if len(stock_news) > 0:
        mean_sent = stock_news['sentiment_combined'].mean()
        pos_pct = (stock_news['sentiment_label'] == 'positive').sum() / len(stock_news) * 100
        neg_pct = (stock_news['sentiment_label'] == 'negative').sum() / len(stock_news) * 100
        print(f"  {stock:6s}: Mean sentiment = {mean_sent:>6.4f}, Positive = {pos_pct:>5.2f}%, Negative = {neg_pct:>5.2f}%")


In [ ]:
# Visualize sentiment distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Sentiment Analysis Results', fontsize=16, fontweight='bold', y=0.995)

# Plot 1: Sentiment classification distribution
ax1 = axes[0, 0]
sentiment_counts = df_sentiment['sentiment_label'].value_counts()
colors = {'positive': 'green', 'negative': 'red', 'neutral': 'gray'}
bars = ax1.bar(sentiment_counts.index, sentiment_counts.values, 
               color=[colors.get(x, 'blue') for x in sentiment_counts.index])
ax1.set_xlabel('Sentiment Label', fontsize=11)
ax1.set_ylabel('Number of Articles', fontsize=11)
ax1.set_title('Sentiment Classification Distribution', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}',
             ha='center', va='bottom', fontsize=10)

# Plot 2: Combined sentiment score distribution
ax2 = axes[0, 1]
ax2.hist(df_sentiment['sentiment_combined'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax2.axvline(df_sentiment['sentiment_combined'].mean(), color='r', linestyle='--', linewidth=2, 
            label=f'Mean ({df_sentiment["sentiment_combined"].mean():.3f})')
ax2.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
ax2.set_xlabel('Sentiment Score (Combined)', fontsize=11)
ax2.set_ylabel('Frequency', fontsize=11)
ax2.set_title('Combined Sentiment Score Distribution', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Comparison of TextBlob vs VADER
ax3 = axes[1, 0]
# Sample for scatter plot if too many points
if len(df_sentiment) > 10000:
    sample_df = df_sentiment.sample(n=10000, random_state=42)
else:
    sample_df = df_sentiment
ax3.scatter(sample_df['sentiment_textblob'], sample_df['sentiment_vader'], 
           alpha=0.3, s=10, color='purple')
ax3.plot([-1, 1], [-1, 1], 'r--', linewidth=1, alpha=0.5, label='Perfect Agreement')
ax3.set_xlabel('TextBlob Sentiment', fontsize=11)
ax3.set_ylabel('VADER Sentiment', fontsize=11)
ax3.set_title('TextBlob vs VADER Sentiment Comparison', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Sentiment by stock
ax4 = axes[1, 1]
sentiment_by_stock = df_sentiment.groupby('stock')['sentiment_combined'].mean().sort_values()
bars = ax4.bar(range(len(sentiment_by_stock)), sentiment_by_stock.values, color='coral')
ax4.set_xticks(range(len(sentiment_by_stock)))
ax4.set_xticklabels(sentiment_by_stock.index, rotation=45, ha='right')
ax4.set_ylabel('Mean Sentiment Score', fontsize=11)
ax4.set_title('Average Sentiment by Stock', fontsize=12, fontweight='bold')
ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (stock, val) in enumerate(sentiment_by_stock.items()):
    ax4.text(i, val, f'{val:.3f}', ha='center', va='bottom' if val > 0 else 'top', fontsize=9)

plt.tight_layout()
plt.savefig('figures/sentiment_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: figures/sentiment_analysis.png")


## 3. Stock Movement Calculation

**Objective:** Calculate daily stock returns to represent stock movements.

**Key Steps:**
- Calculate daily percentage change in closing prices
- Handle missing trading days
- Analyze return distribution


In [ ]:
# Calculate daily stock returns
print("="*80)
print("STOCK MOVEMENT CALCULATION")
print("="*80)

stock_returns = {}

for stock, df_stock in stock_data.items():
    # Calculate daily returns (percentage change)
    df_stock['daily_return'] = df_stock['Close'].pct_change() * 100  # Convert to percentage
    
    # Create returns dataframe with date
    returns_df = pd.DataFrame({
        'date': df_stock.index,
        'date_only': df_stock.index.date,
        'daily_return': df_stock['daily_return'],
        'close_price': df_stock['Close']
    })
    
    stock_returns[stock] = returns_df
    
    # Statistics
    returns_clean = returns_df['daily_return'].dropna()
    print(f"\n{stock}:")
    print(f"  Total trading days: {len(returns_df):,}")
    print(f"  Mean daily return: {returns_clean.mean():.4f}%")
    print(f"  Std dev: {returns_clean.std():.4f}%")
    print(f"  Min return: {returns_clean.min():.4f}%")
    print(f"  Max return: {returns_clean.max():.4f}%")
    print(f"  Positive days: {(returns_clean > 0).sum():,} ({(returns_clean > 0).sum()/len(returns_clean)*100:.2f}%)")
    print(f"  Negative days: {(returns_clean < 0).sum():,} ({(returns_clean < 0).sum()/len(returns_clean)*100:.2f}%)")

print(f"\n✓ Daily returns calculated for {len(stock_returns)} stocks")


In [ ]:
# Visualize stock returns distribution
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Daily Stock Returns Distribution', fontsize=16, fontweight='bold', y=0.995)
axes = axes.flatten()

for idx, (stock, returns_df) in enumerate(stock_returns.items()):
    ax = axes[idx]
    returns_clean = returns_df['daily_return'].dropna()
    
    ax.hist(returns_clean, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax.axvline(returns_clean.mean(), color='r', linestyle='--', linewidth=2, 
              label=f'Mean ({returns_clean.mean():.3f}%)')
    ax.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    ax.set_xlabel('Daily Return (%)', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.set_title(f'{stock} Daily Returns', fontsize=11, fontweight='bold')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('figures/stock_returns_distribution.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: figures/stock_returns_distribution.png")


## 4. Correlation Analysis

**Objective:** Aggregate daily sentiments and calculate correlation between news sentiment and stock returns.

**Key Steps:**
- Aggregate daily sentiment scores (average if multiple articles per day)
- Match sentiment scores to stock returns by date and stock
- Calculate Pearson correlation coefficient
- Perform statistical significance testing
- Analyze correlation patterns


In [ ]:
# Aggregate daily sentiments
print("="*80)
print("AGGREGATING DAILY SENTIMENTS")
print("="*80)

# Group by date and stock, calculate average sentiment
daily_sentiments = df_sentiment.groupby(['date_only', 'stock']).agg({
    'sentiment_combined': 'mean',
    'sentiment_textblob': 'mean',
    'sentiment_vader': 'mean',
    'headline': 'count'
}).reset_index()

daily_sentiments.columns = ['date_only', 'stock', 'avg_sentiment_combined', 
                           'avg_sentiment_textblob', 'avg_sentiment_vader', 'article_count']

print(f"\nDaily sentiment aggregation:")
print(f"  Total date-stock combinations: {len(daily_sentiments):,}")
print(f"  Days with multiple articles: {(daily_sentiments['article_count'] > 1).sum():,}")
print(f"  Average articles per day-stock: {daily_sentiments['article_count'].mean():.2f}")
print(f"  Max articles in a single day-stock: {daily_sentiments['article_count'].max():,}")

# Show examples
print(f"\nSample of daily aggregated sentiments:")
print(daily_sentiments.head(10).to_string(index=False))


In [ ]:
# Merge sentiment and stock returns data
print("\n" + "="*80)
print("MERGING SENTIMENT AND STOCK RETURNS")
print("="*80)

correlation_data = {}

for stock in STOCKS:
    # Get sentiment data for this stock
    stock_sentiment = daily_sentiments[daily_sentiments['stock'] == stock].copy()
    
    # Get returns data for this stock
    stock_return = stock_returns[stock].copy()
    
    # Merge on date_only
    merged = pd.merge(
        stock_sentiment,
        stock_return[['date_only', 'daily_return']],
        on='date_only',
        how='inner'
    )
    
    # Remove rows with missing returns or sentiment
    merged = merged.dropna(subset=['daily_return', 'avg_sentiment_combined'])
    
    correlation_data[stock] = merged
    
    print(f"\n{stock}:")
    print(f"  Matched date-stock pairs: {len(merged):,}")
    print(f"  Date range: {merged['date_only'].min()} to {merged['date_only'].max()}")
    print(f"  Coverage: {len(merged)/len(stock_return)*100:.2f}% of trading days have news")

print(f"\n✓ Data merged for {len(correlation_data)} stocks")


In [ ]:
# Calculate correlation coefficients
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlation_results = {}

for stock, merged_df in correlation_data.items():
    if len(merged_df) < 2:
        print(f"\n{stock}: Insufficient data for correlation (need at least 2 data points)")
        continue
    
    # Calculate Pearson correlation
    sentiment = merged_df['avg_sentiment_combined']
    returns = merged_df['daily_return']
    
    # Remove any remaining NaN values
    valid_mask = ~(sentiment.isna() | returns.isna())
    sentiment_clean = sentiment[valid_mask]
    returns_clean = returns[valid_mask]
    
    if len(sentiment_clean) < 2:
        print(f"\n{stock}: Insufficient valid data for correlation")
        continue
    
    # Pearson correlation
    corr_coef, p_value = pearsonr(sentiment_clean, returns_clean)
    
    # Calculate R-squared
    r_squared = corr_coef ** 2
    
    correlation_results[stock] = {
        'correlation': corr_coef,
        'p_value': p_value,
        'r_squared': r_squared,
        'n_observations': len(sentiment_clean),
        'significant': p_value < 0.05
    }
    
    print(f"\n{stock}:")
    print(f"  Pearson Correlation: {corr_coef:>7.4f}")
    print(f"  R-squared:           {r_squared:>7.4f}")
    print(f"  P-value:             {p_value:>7.4e}")
    print(f"  Observations:        {len(sentiment_clean):>7,}")
    print(f"  Significant (p<0.05): {'Yes' if p_value < 0.05 else 'No'}")

# Summary
print(f"\n" + "="*80)
print("CORRELATION SUMMARY")
print("="*80)
if correlation_results:
    avg_corr = np.mean([r['correlation'] for r in correlation_results.values()])
    significant_count = sum(1 for r in correlation_results.values() if r['significant'])
    print(f"Average correlation across stocks: {avg_corr:.4f}")
    print(f"Stocks with significant correlation (p<0.05): {significant_count}/{len(correlation_results)}")


In [ ]:
# Visualize correlations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Correlation Analysis: News Sentiment vs Stock Returns', fontsize=16, fontweight='bold', y=0.995)
axes = axes.flatten()

for idx, stock in enumerate(STOCKS):
    ax = axes[idx]
    
    if stock in correlation_data and len(correlation_data[stock]) > 0:
        merged_df = correlation_data[stock]
        
        # Scatter plot
        ax.scatter(merged_df['avg_sentiment_combined'], merged_df['daily_return'], 
                  alpha=0.5, s=20, color='steelblue')
        
        # Add trend line
        z = np.polyfit(merged_df['avg_sentiment_combined'], merged_df['daily_return'], 1)
        p = np.poly1d(z)
        ax.plot(merged_df['avg_sentiment_combined'], p(merged_df['avg_sentiment_combined']), 
               "r--", alpha=0.8, linewidth=2, label=f"Trend line")
        
        # Add correlation info
        if stock in correlation_results:
            corr = correlation_results[stock]['correlation']
            p_val = correlation_results[stock]['p_value']
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
            ax.text(0.05, 0.95, f'r = {corr:.3f}{sig}\np = {p_val:.3e}', 
                   transform=ax.transAxes, fontsize=10,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        ax.set_xlabel('Average Daily Sentiment Score', fontsize=10)
        ax.set_ylabel('Daily Return (%)', fontsize=10)
        ax.set_title(f'{stock} - Sentiment vs Returns', fontsize=11, fontweight='bold')
        ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)
        ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'Insufficient data', ha='center', va='center', 
               transform=ax.transAxes, fontsize=12)
        ax.set_title(f'{stock}', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('figures/sentiment_returns_correlation.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: figures/sentiment_returns_correlation.png")


In [ ]:
# Create correlation summary visualization
if correlation_results:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Correlation Analysis Summary', fontsize=16, fontweight='bold', y=1.0)
    
    # Plot 1: Correlation coefficients by stock
    ax1 = axes[0]
    stocks = list(correlation_results.keys())
    correlations = [correlation_results[s]['correlation'] for s in stocks]
    p_values = [correlation_results[s]['p_value'] for s in stocks]
    colors = ['green' if p < 0.05 else 'red' for p in p_values]
    
    bars = ax1.bar(range(len(stocks)), correlations, color=colors, alpha=0.7)
    ax1.set_xticks(range(len(stocks)))
    ax1.set_xticklabels(stocks)
    ax1.set_ylabel('Pearson Correlation Coefficient', fontsize=11)
    ax1.set_title('Correlation: Sentiment vs Stock Returns by Stock', fontsize=12, fontweight='bold')
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for i, (corr, p_val) in enumerate(zip(correlations, p_values)):
        sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
        ax1.text(i, corr, f'{corr:.3f}{sig}', ha='center', 
                va='bottom' if corr > 0 else 'top', fontsize=9)
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='green', alpha=0.7, label='Significant (p<0.05)'),
        Patch(facecolor='red', alpha=0.7, label='Not Significant')
    ]
    ax1.legend(handles=legend_elements, loc='best', fontsize=9)
    
    # Plot 2: R-squared values
    ax2 = axes[1]
    r_squared = [correlation_results[s]['r_squared'] for s in stocks]
    bars2 = ax2.bar(range(len(stocks)), r_squared, color='steelblue', alpha=0.7)
    ax2.set_xticks(range(len(stocks)))
    ax2.set_xticklabels(stocks)
    ax2.set_ylabel('R-squared', fontsize=11)
    ax2.set_title('R-squared: Variance Explained', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for i, r2 in enumerate(r_squared):
        ax2.text(i, r2, f'{r2:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('figures/correlation_summary.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Saved: figures/correlation_summary.png")


In [ ]:
# Additional analysis: Lagged correlation (sentiment today vs returns tomorrow)
print("\n" + "="*80)
print("LAGGED CORRELATION ANALYSIS")
print("="*80)
print("Analyzing correlation between sentiment and next-day returns...")

lagged_correlation_results = {}

for stock in STOCKS:
    if stock not in correlation_data or len(correlation_data[stock]) < 2:
        continue
    
    merged_df = correlation_data[stock].copy()
    merged_df = merged_df.sort_values('date_only').reset_index(drop=True)
    
    # Shift returns by 1 day (sentiment today -> returns tomorrow)
    merged_df['next_day_return'] = merged_df['daily_return'].shift(-1)
    
    # Remove NaN from shifted column
    lagged_df = merged_df[['avg_sentiment_combined', 'next_day_return']].dropna()
    
    if len(lagged_df) < 2:
        continue
    
    corr_lag, p_val_lag = pearsonr(lagged_df['avg_sentiment_combined'], lagged_df['next_day_return'])
    
    lagged_correlation_results[stock] = {
        'correlation': corr_lag,
        'p_value': p_val_lag,
        'n_observations': len(lagged_df)
    }
    
    print(f"\n{stock}:")
    print(f"  Lagged Correlation (sentiment -> next day return): {corr_lag:>7.4f}")
    print(f"  P-value: {p_val_lag:>7.4e}")
    print(f"  Significant (p<0.05): {'Yes' if p_val_lag < 0.05 else 'No'}")

if lagged_correlation_results:
    print(f"\n" + "="*80)
    print("LAGGED CORRELATION SUMMARY")
    print("="*80)
    avg_lag_corr = np.mean([r['correlation'] for r in lagged_correlation_results.values()])
    sig_lag_count = sum(1 for r in lagged_correlation_results.values() if r['p_value'] < 0.05)
    print(f"Average lagged correlation: {avg_lag_corr:.4f}")
    print(f"Stocks with significant lagged correlation: {sig_lag_count}/{len(lagged_correlation_results)}")


## Summary and Key Findings

**Key Insights from Correlation Analysis:**

1. **Sentiment Analysis Results:**
   - [To be filled after running the notebook]

2. **Stock Returns:**
   - [To be filled after running the notebook]

3. **Correlation Findings:**
   - [To be filled after running the notebook]

4. **Statistical Significance:**
   - [To be filled after running the notebook]

5. **Lagged Correlation:**
   - [To be filled after running the notebook]

**Next Steps:**
- Interpret correlation results in business context
- Consider additional factors (volume, volatility, market conditions)
- Explore non-linear relationships if linear correlation is weak


In [ ]:
# Ensure figures directory exists
figures_dir = 'figures'
if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)
    print(f"Created {figures_dir}/ directory")
else:
    print(f"Using existing {figures_dir}/ directory")
